Calculate ensemble of sea level projections

In [ ]:
import numpy as np
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd

import sys
sys.path.append('../src')

from alldata import AllData
from ensemble import EnsembleMember,FullEnsemble2

np.set_printoptions(precision=2)
%matplotlib notebook
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

Read data

In [ ]:
ad = AllData()
ad.year0 = 1951
ad.usefanom = True
ad.gather(update=False)

#Omit invalid ism
ad.ds = ad.ds.drop_sel(ism='PISM_DMI')

#Optionally subsection of models for rapid calculation
#ad.ds = ad.ds.isel(ism=slice(0,2),esm=slice(0,2))

fe = FullEnsemble2(ad)
fe.nonlin = 'cutoff'
fe.gather(force_update=False)

Figure settings

In [ ]:
plt.style.use('../src/style_paper')
mpl.rc('figure.subplot',left=.15,bottom=.2,wspace=.15)

alpha = .3 #Transparency of uncertainty range
perc = [17,83] #Percentiles of uncertainty range

Make and save figure

In [ ]:
fig,ax = plt.subplots(1,len(fe.ssp),sharex=True,sharey='row',figsize=(7,3))

for s,ss in enumerate(fe.ssp.values):
    #print(np.sum(~np.isnan(fe.gamma[0,:,:].values)))    
    
    #Plot no feedback
    ax[s].plot(fe.time,100*np.nanmedian(fe.slr[0,:,s,:,:,:],axis=(0,1,2)),c='.5',lw=2,label='No feedback')
    ax[s].fill_between(fe.time,100*np.nanpercentile(fe.slr[0,:,s,:,:,:],perc[0],axis=(0,1,2)),100*np.nanpercentile(fe.slr[0,:,s,:,:,:],perc[1],axis=(0,1,2)),color='.5',alpha=alpha)#,label=perc)

    #Plot with feedback
    ax[s].plot(fe.time,100*np.nanmedian(fe.slr[1,:,s,:,:,:],axis=(0,1,2)),c=ad.scol[ss],lw=2,label='Feedback')
    ax[s].fill_between(fe.time,100*np.nanpercentile(fe.slr[1,:,s,:,:,:],perc[0],axis=(0,1,2)),100*np.nanpercentile(fe.slr[1,:,s,:,:,:],perc[1],axis=(0,1,2)),color=ad.scol[ss],alpha=alpha)

    ax[s].set_title(f'SSP-{ss}')
    
    print(ss,np.nanmedian(fe.slr[1,:,s,:,:,:],axis=(0,1,2))[-1]/(np.nanmedian(fe.slr[0,:,s,:,:,:],axis=(0,1,2))[-1]))
    #print(ss,'SLR_amp','ar',np.nanmedian(fe.slr[2,s,:,:,:],axis=(0,1))[-1]/(np.nanmedian(fe.slr[0,s,:,:,:],axis=(0,1))[-1]))

for a,aa in enumerate(['nf','wf']):
    for s,ss in enumerate(fe.ssp.values):
        print(aa,ss,100*np.nanmedian(fe.slr[a,:,s,:,:,2017-fe.year0]-fe.slr[a,:,s,:,:,1979-fe.year0]))
    
ax[0].legend(loc='upper left')
ax[0].set_ylabel('Cumulative sea-level rise [cm]')
#ax[0,0].set_ylabel('Before recalibration',fontsize=9)
#ax[1,0].set_ylabel('After recalibration',fontsize=9)
ax[1].set_xlabel('Time [years]')

#ax[1].set_xlim([1979,2017])
#ax[1].set_ylim([0,2])

plt.savefig(f'../draftfigs/full_ensemble2_{fe.nonlin}_{fe.year0}.png',dpi=450,facecolor='w',transparent=False)
plt.show()